# Question answering with LLM

In [ ]:
#|default_exp qa.ablation

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#|export
import magentic
from pydantic import BaseModel, Field

from bellek.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|export
DEFAULT_MODEL = magentic.OpenaiChatModel("gpt-3.5-turbo", temperature=0.1)

In [ ]:
#|export

USER_PROMPT = """The context information is provided below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the question.
{question}
"""

In [ ]:
#|export

SYSTEM_PROMPT = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge."""

class QuestionAnsweringResultStandard(BaseModel):
    """Data model for answering the question."""

    answer: str = Field(description="The answer to the question in 2-4 words.")


@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT), 
    magentic.UserMessage(USER_PROMPT),
    model=DEFAULT_MODEL,
)
def answer_question_standard(
    context: str,
    question: str,
) -> QuestionAnsweringResultStandard: ...

In [ ]:
#|export

SYSTEM_PROMPT_REASONING = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge. Always provide clear and logical step-by-step reasoning in your answers."""

class QuestionAnsweringResultCOT(BaseModel):
    """Data model for answering the question."""

    reasoning: str = Field(description="Step-by-step reasoning for the answer.")
    answer: str = Field(description="The answer to the question in 2-4 words.")


@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_REASONING), 
    magentic.UserMessage(USER_PROMPT),
    model=DEFAULT_MODEL,
)
def answer_question_cot(
    context: str,
    question: str,
) -> QuestionAnsweringResultCOT: ...

In [ ]:
#|export

SYSTEM_PROMPT_WITH_TRIPLETS = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge.

Before answering the question, first, you extract relevant entity-relation-entity triplets from the context. Then, you answer the question based on the triplets. For instance, 

# Example
Context: "Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011."

Question: "Who is the current mayor of city Glenhis Hernández was born?"

Triplets: "Glenhis Hernández (Athlete) | born on | October 7, 1990
Glenhis Hernández (Athlete) | birth place | Havana
Glenhis Hernández (Athlete) | specializes in | taekwondo
Glenhis Hernández (Athlete) | won | 2013 World Champion title (Middleweight)
Marta Hernández Romero (Politician) | serves as | mayor of Havana
Marta Hernández Romero (Politician) | holds | the position of 'President of the People's Power Provincial Assembly'
Marta Hernández Romero (Politician) | elected on | March 5, 2011."

Answer: "Marta Hernández Romero"
"""

class _QuestionAnsweringResultCTE(BaseModel):
    """Data model for answering the question."""

    triplets: list[str] = Field(description="A list of entity-relation-entity triplets extracted from the context.")
    answer: str = Field(description="The answer to the question in 2-4 words.")

class QuestionAnsweringResultCTE(_QuestionAnsweringResultCTE):
    reasoning: str = ""

@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_WITH_TRIPLETS), 
    magentic.UserMessage(USER_PROMPT),
    model=DEFAULT_MODEL,
)
def _answer_question_cte(
    context: str,
    question: str,
) -> _QuestionAnsweringResultCTE: ...

def answer_question_cte(
    context: str,
    question: str,
) -> QuestionAnsweringResultCTE: 
    result = _answer_question_cte(context, question)
    return QuestionAnsweringResultCTE(triplets=result.triplets, answer=result.answer)

In [ ]:
#|export

SYSTEM_PROMPT_REASONING_WITH_TRIPLETS = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on context the information given, without drawing on prior knowledge. Always provide clear and logical step-by-step reasoning in your answers.

Before answering the question, first, you extract relevant entity-relation-entity triplets from the context. Then, you answer the question based on the triplets. For instance, 

# Example
Context: "Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011."

Question: "Who is the current mayor of city Glenhis Hernández was born?"

Triplets: "Glenhis Hernández (Athlete) | born on | October 7, 1990
Glenhis Hernández (Athlete) | birth place | Havana
Glenhis Hernández (Athlete) | specializes in | taekwondo
Glenhis Hernández (Athlete) | won | 2013 World Champion title (Middleweight)
Marta Hernández Romero (Politician) | serves as | mayor of Havana
Marta Hernández Romero (Politician) | holds | the position of 'President of the People's Power Provincial Assembly'
Marta Hernández Romero (Politician) | elected on | March 5, 2011."

Reasoning: "Glenhis Hernández (Athlete) | birth place | Havana
This indicates that Glenhis Hernández was born in Havana.

Marta Hernández Romero (Politician) | serves as | mayor of Havana
This states that Marta Hernández Romero is the mayor of Havana.

From these triplets, we conclude that Marta Hernández Romero is the mayor of Havana."

Answer: "Marta Hernández Romero"
"""

class QuestionAnsweringResultCTECOT(BaseModel):
    """Data model for answering the question."""

    triplets: list[str] = Field(description="A list of entity-relation-entity triplets extracted from the context.")
    reasoning: str = Field(description="Step-by-step reasoning for the answer.")
    answer: str = Field(description="The answer to the question in 2-4 words.")


@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_REASONING_WITH_TRIPLETS), 
    magentic.UserMessage(USER_PROMPT),
    model=DEFAULT_MODEL,
)
def answer_question_cte_cot(
    context: str,
    question: str,
) -> QuestionAnsweringResultCTECOT: ...

In [ ]:
context = """Sings America is an album released by David Hasselhoff in August 2004 (see 2004 in music). The album contains covers of songs originally made famous by artists such as Elvis Presley, The Beach Boys, Glen Campbell, Burt Bacharach and Madonna. The German release contains a bonus track, "More Than Words Can Say", which is the only original Hasselhoff composition on the album (written in conjunction with Wade Hubbard and Glenn Morrow)."""
question = "Who performed Sings America?"
answer = "David Hasselhoff"

In [ ]:
result = answer_question_standard(context=context, question=question)
print(result.answer)

David Hasselhoff


In [ ]:
result = answer_question_cot(context=context, question=question)
print(result.answer)
print()
print(result.reasoning)

David Hasselhoff

The album 'Sings America' was released by David Hasselhoff in August 2004, as mentioned in the context information.


In [ ]:
result = answer_question_cte(context=context, question=question)
print(result.answer)
print()
for triplet in result.triplets:
    print(triplet)
print()
print(result.reasoning)

David Hasselhoff

David Hasselhoff (Musician) | released | Sings America album
Sings America album | contains | covers of songs by Elvis Presley, The Beach Boys, Glen Campbell, Burt Bacharach, and Madonna
Sings America album | contains | bonus track 'More Than Words Can Say' (original Hasselhoff composition)
'More Than Words Can Say' | written by | David Hasselhoff, Wade Hubbard, Glenn Morrow




In [ ]:
result = answer_question_cte_cot(context=context, question=question)
print(result.answer)
print()
for triplet in result.triplets:
    print(triplet)
print()
print(result.reasoning)

David Hasselhoff

David Hasselhoff (Musician) | released | Sings America album
David Hasselhoff (Musician) | released in | August 2004
David Hasselhoff (Musician) | album contains | covers of songs by various artists like Elvis Presley, The Beach Boys, Glen Campbell, Burt Bacharach, and Madonna
David Hasselhoff (Musician) | composed | bonus track 'More Than Words Can Say' in conjunction with Wade Hubbard and Glenn Morrow

David Hasselhoff released the album 'Sings America' in August 2004, which contains covers of songs by various artists. He also composed the bonus track 'More Than Words Can Say' in conjunction with Wade Hubbard and Glenn Morrow.


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()